# Report for Ratings-Results

In [34]:
import report

report.create_report_files(
    "../keys/gs-on-a-budget-firebase-adminsdk-63ws0-509fffaf5f.json",
    "ratings.xlsx",
    "pairs.xlsx",
)

In [35]:
import pandas as pd

pairs = pd.read_excel("./pairs.xlsx")
pairs

,Unnamed: 0,rotation,model_2,model_1,size,dataset,position,n_ratings,high_detail,id
0,0,"[0.9961567660501535, 0.008693328396189522, 0.0...",splats/mcmc-truck-low-1.ksplat,splats/default-truck-low-1.ksplat,low,truck,"[0.58, 0.0, 0.65]",0,False,05bDTzubIAQcN7s6AkHs
1,1,"[0.9961567660501535, 0.008693328396189522, 0.0...",splats/mcmc-truck-extended-1.ksplat,splats/default-truck-extended-1.ksplat,extended,truck,"[0.58, 0.0, 0.65]",0,True,Lk507yRU9Gcvoqr66PTD
2,2,"[0.9961567660501535, 0.008693328396189522, 0.0...",splats/mcmc-truck-medium-1.ksplat,splats/default-truck-medium-1.ksplat,medium,truck,"[0.58, 0.0, 0.65]",0,False,bzNWpl95kMDv3tzBcsHr
3,3,"[0.9961567660501535, 0.008693328396189522, 0.0...",splats/mcmc-truck-high-1.ksplat,splats/default-truck-high-1.ksplat,high,truck,"[0.58, 0.0, 0.65]",0,True,gLnu0rNhjibQDbl0lUjC


In [36]:
ratings = pd.read_excel("./ratings.xlsx")
ratings

,Unnamed: 0,rating_1,user_id,pair_id,rating_2,id
0,0,4,e2b916b6-3061-4065-9db3-13ef6e95a46e,Lk507yRU9Gcvoqr66PTD,5,4o32zltKIbKPjOxv9Clb
1,1,5,ad545466-8e7e-40f3-b1dd-507bce895d55,Lk507yRU9Gcvoqr66PTD,3,6ey445gPh4DWmRZNVd1V
2,2,4,e7dcf150-4a7e-4959-96ba-8da47bfe2084,05bDTzubIAQcN7s6AkHs,2,8P5TDkHbpu6kFpNV8D0N
3,3,5,9913cbbf-2caf-4a24-8c10-33c7ea6b8d91,Lk507yRU9Gcvoqr66PTD,4,AGuj13pB4sx2Bk3NBEfE
4,4,5,e2b916b6-3061-4065-9db3-13ef6e95a46e,05bDTzubIAQcN7s6AkHs,3,DwqHljyNMF5ghD6mKa59
5,5,2,e2b916b6-3061-4065-9db3-13ef6e95a46e,Lk507yRU9Gcvoqr66PTD,4,F3hX6Kpk6HPcsMmdhFAH
6,6,2,e2b916b6-3061-4065-9db3-13ef6e95a46e,Lk507yRU9Gcvoqr66PTD,3,FZ99uZX7EZ4dim0KvYc0
7,7,3,9913cbbf-2caf-4a24-8c10-33c7ea6b8d91,05bDTzubIAQcN7s6AkHs,5,JvhsKqbqzsVBy8Zs5aE4
8,8,4,7fae8721-2af1-487e-8749-b07f1965c2e9,05bDTzubIAQcN7s6AkHs,5,Q94oEO23Zol29DoV2GPe
9,9,4,e2b916b6-3061-4065-9db3-13ef6e95a46e,05bDTzubIAQcN7s6AkHs,2,UY0Dnp0MuursUFjz4Tui


In [37]:
ratings_count_computed = ratings.groupby("pair_id")["id"].count()

In [38]:
# sanity check to see if ratings and n_ratings actually match up
for pair_id, n_ratings in ratings_count_computed.items():
    pair = pairs[pairs["id"] == pair_id]
    if pair.shape[0] != 1:
        print(f'ERROR: pair "{pair_id}" occurs "{pair.shape[0]}" times')
    db_n_ratings = pair["n_ratings"].values[0]
    if n_ratings != db_n_ratings:
        print(
            f'ERROR: pair "{pair_id}" has {n_ratings} but ratings tracker only has {db_n_ratings}'
        )

ERROR: pair "05bDTzubIAQcN7s6AkHs" has 7 but ratings tracker only has 0
ERROR: pair "Lk507yRU9Gcvoqr66PTD" has 7 but ratings tracker only has 0
